In [240]:
import numpy as np
import scipy.stats as sps
import scipy.linalg as linalg
import scipy.optimize as opt

Введём параметры нашей системы. Мы будем рассматривать вектора размерности $n$, в которых не более $k$ ненулевых компонент. Мы будем отображать эти вектора в $m$-мерное пространство. $m$ я прикинул из статьи как $\sim k \log_2 \frac{n}{k}$.

In [241]:
n, m, k = 100, 30, 7
a = sps.norm.rvs(size=(m, n))

Следующая функция генерирует разреженные вектора:

In [242]:
def gen():
    x = np.random.rand(n)
    random_mask = np.concatenate((np.zeros(n - k), np.ones(k)))
    np.random.shuffle(random_mask)
    return x * random_mask

Сжатие --- это просто домножение на матрицу

In [243]:
def compress(x):
    return a @ x

Я ищу решение в линейном пространстве решений системы $y = Ax$, минимизируя $L_1$-норму $x$.

In [244]:
def decompress(y):
    x0 = linalg.lstsq(a, y)[0]
    linspace = linalg.null_space(a)
    fun = lambda c: scipy.linalg.norm(linspace @ c + x0, ord=1)
    res = opt.minimize(fun=fun, x0=np.zeros(n - m))
    return linspace @ res.x + x0

Посмотрим на количество ошибок в декодировании:

In [249]:
failures = 0
for i in range(100):
    x = gen()
    y = compress(x)
    res = decompress(y)
    res1 = res.copy()
    res1 -= x
    res1[res1 < 1e-5] = 0
    if np.linalg.norm(res1, ord=0) > 0:
        failures += 1
print(failures)

36


Пока не слишком удачный результат. Попробуем другое распределение.

In [254]:
a = sps.bernoulli.rvs(p=1/2, size=(m, n))

In [255]:
failures = 0
for i in range(100):
    x = gen()
    y = compress(x)
    res = decompress(y)
    res1 = res.copy()
    res1 -= x
    res1[res1 < 1e-5] = 0
    if np.linalg.norm(res1, ord=0) > 0:
        failures += 1
print(failures)

3


Гораздо лучше! Но всё равно, я полагаю, дело в том, что я выбрал $m$ недостаточно большим, а если брать его ещё больше, то оно станет совсем уж близко к $n$. Нужно научиться быстрее оптимизировать $L_1$-норму, потому что иначе это работает слишком медленно.